## Chattanooga Police Department Incident Project
### Part 1


## Table of Contents
### Section 1 - Current Notebook
- [Introduction](#intro)
- [Part I - Exploration of Data](#Exploration)
- [Part II - Cleaning](#cleaning)

### Section 2 - Separate Notebook
- [Part III - Analysis](#analysis)
- [Part IV - Visualization](#visualization)


<a id='intro'></a>
### Introduction

This project is based on information from the open data on Chattanooga.gov.  The data below comes from police incidents and will be analysed to see what we can learn about crime and the neighborhoods in Chattanooga.

This notebook contains the exploration and cleaning process for the project.  The Analysis and Visualization of the findings will be contained in a separate notebook. Some of the visualizations will be completed using Tableau in order to better map the data on a physical GIS map.

During the exploration and cleaning phases it became abundantly clear that this data is not only messy but not tidy.  There are numerous spelling, organzational, and accuracy issues.  The city of Chattanooga was misspelled so many times it was frustrating. Neighborhoods within the city of Chattanooga were used as the city's name which caused problems as well. Within the cleaning section there is a breakdown of what is to be cleaned and how each problem will be taken on.


In [1]:
#import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns

In [2]:
#Read in data set
df = pd.read_csv('Police__Incident_Data.csv')

In [3]:
df.head()

,Address,City,State,Zip,Date_Incident,Jurisdiction,Incident_Tract,UCR_Incident_Code,Incident_Description,Incident_Type,Case_Number,Case_Status,Case_Status_Description,Latitude,Longitude,Location
0,2200 Hamilton Place Blvd,Chattanooga,TN,37421,08/29/2017 11:20:00 AM,TN0330100,C7,23C,Shoplifting,Part 1 Crimes,17-084827,3,Pending/Possible Leads,35.039741,-85.155184,"(35.039741454, -85.155184058)"
1,2500 Mccrae St,Chattanooga,TN,37406,07/29/2017 11:01:00 PM,TN0330100,B2,91Z,NaN,NaN,17-073656,3,Pending/Possible Leads,35.058561,-85.252378,"(35.058561443, -85.252378156)"
2,1100 THOMAS LN.,Chattanooga,TN,37343-,11/08/2015 10:00:00 AM,TN0330100,A1,220,Burglary/Breaking And Entering,Part 1 Crimes,15-112814,3,Pending/Possible Leads,35.142962,-85.247589,"(35.142962, -85.247589)"
3,3400 VINEWOOD DR,Chattanooga,TN,37406--,02/02/2015 11:38:00 PM,TN0330100,E1,000,Misc Report,Other Incidents,15-010660,NaN,NaN,35.074546,-85.239625,"(35.074546, -85.239625)"
4,2800 Dodson Ave,Chattanooga,TN,37415-,04/08/2015 12:18:00 PM,TN0330100,E2,000,Misc Report,Other Incidents,15-032003,NaN,NaN,35.069329,-85.250511,"(35.069329, -85.250511)"


<a id='Exploration'></a>
### Exploration of Data

In [4]:
df.info()
#It Appears that there are 241338 events

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241338 entries, 0 to 241337
Data columns (total 16 columns):
Address                    241337 non-null object
City                       241243 non-null object
State                      241338 non-null object
Zip                        241242 non-null object
Date_Incident              241338 non-null object
Jurisdiction               241338 non-null object
Incident_Tract             240025 non-null object
UCR_Incident_Code          241338 non-null object
Incident_Description       182454 non-null object
Incident_Type              182454 non-null object
Case_Number                241338 non-null object
Case_Status                176913 non-null object
Case_Status_Description    176898 non-null object
Latitude                   200795 non-null float64
Longitude                  200795 non-null float64
Location                   200795 non-null object
dtypes: float64(2), object(14)
memory usage: 29.5+ MB


In [5]:
df.Jurisdiction.value_counts()
#There is only one jurisdiction - CPD

TN0330100    241338
Name: Jurisdiction, dtype: int64

In [6]:
df.City.value_counts()

Chattanooga        211434
CHATTANOOGA         25147
Hixson               3137
HIXSON                301
chattanooga           240
Ooltewah              148
CHATT                  72
Chattanoga             61
OOLTEWAH               40
CHATTANOGOA            39
Chattanooga,           32
Chattanoogaq           31
CH                     28
CHATANOOGA             27
Lupton City            26
CHATTANOOG             23
East Ridge             21
RED BANK               15
Chatanooga             15
CHATTNAOOGA            14
Chatt                  13
Chattannoga            13
hixson                 11
Chatttanooga           11
EAST RIDGE             11
CHATTANOOA             11
CHATTTANOOGA           11
CHATTANOGA             10
Harrison               10
Chattnooga              9
                    ...  
Chattnaooga             1
SIGNAL MTN.             1
CHATTANOOGSA            1
RINGGOLD                1
Spencer                 1
Chattano0ga             1
CHATTANOOGA,,,,         1
NORWICH     

In [7]:
#See when the first and last incident reported is by sorting by date and looking at first 5 and last 5
df.Date_Incident.sort_values()
#This returned 1-1-15 as first event and 12-31-18 as final event

132390    01/01/2015 01:00:00 PM
159153    01/01/2015 01:00:00 PM
164616    01/01/2015 01:04:00 PM
152698    01/01/2015 01:12:00 PM
133007    01/01/2015 01:22:00 PM
87831     01/01/2015 01:25:00 PM
132937    01/01/2015 01:27:00 PM
181067    01/01/2015 01:31:00 PM
52192     01/01/2015 01:32:00 PM
109999    01/01/2015 01:38:00 PM
163612    01/01/2015 01:38:00 PM
46773     01/01/2015 01:56:00 PM
115307    01/01/2015 02:00:00 PM
152445    01/01/2015 02:00:00 PM
104027    01/01/2015 02:00:00 PM
151183    01/01/2015 02:02:00 PM
130831    01/01/2015 02:11:00 PM
99243     01/01/2015 02:13:00 PM
122752    01/01/2015 02:14:00 PM
138734    01/01/2015 02:22:00 PM
173474    01/01/2015 02:37:00 PM
151196    01/01/2015 02:40:00 PM
169838    01/01/2015 02:43:00 PM
72320     01/01/2015 02:43:00 PM
169218    01/01/2015 02:47:00 PM
160265    01/01/2015 03:00:00 PM
180885    01/01/2015 03:03:00 PM
148965    01/01/2015 03:10:00 PM
141332    01/01/2015 03:25:00 PM
54881     01/01/2015 03:30:00 PM
          

In [8]:
#How many types of UCR_Incident Codes, Incident Tract, incident description and type are there.
df.Incident_Tract.value_counts()
#Incident Tract is going to be the Zone or District the call or incident occurred in

A1            12305
A3            11187
C2             9211
C9             7902
C5             7269
C7             7243
A5             6809
B9             6643
A4             6627
B6             6622
B7             6322
C8             6281
A2             6041
A12            5951
C3             5687
C6             5670
B4             5551
C4             5486
B2             5399
B8             4943
C1             4915
A9             4793
D2             4775
B3             4594
A13            4560
A7             4473
B5             4387
G5             3767
A10            3721
B1             3609
              ...  
C11            1336
9 East           36
7 East           34
1 East           33
5 East           28
ERPD             27
3 East           24
8 East           20
4 East           19
6 East           19
2 East           18
10 East           8
OUTSIDE           5
3 West            5
Collegedal        4
1 West            4
4 West            3
HCER              3
ZONE 2            3


In [9]:
#UCR_Incident Code
df.UCR_Incident_Code.value_counts()
#This is the Uniform Crime Reporting Code, 94 unique codes present

91Z        58686
000        57735
290        11129
13B        10835
23C         9645
23F         8896
35A         7037
220         6652
23H         6098
ALB         5121
13A         4736
240         4726
23D         4710
90Z         4379
35B         3853
ALR         3542
23G         2634
26A         2408
PLT         1855
26B         1795
CIT         1675
520         1671
120         1478
90E         1470
90D         1387
250         1328
90J         1274
90C         1149
CVM         1144
MSP         1055
           ...  
AOA           59
90G           55
FIR           53
ANM           49
36B           48
39C           33
39B           31
210           25
UDI           24
40B           20
AST           19
26G           16
ANC           16
11C           16
720           16
36A           14
90B           10
DOC            9
64A            8
09C            7
PRL            5
510            5
26D            5
FID            4
09B            4
09D            3
90H            3
39A           

In [10]:
#Incident Description
df.Incident_Description.value_counts()
#Plain Language Description of Crimes, 83 different types

Misc Report                                    57735
Destructive/Damage/Vandalism Of Property       11129
Simple Assault                                 10835
Shoplifting                                     9645
Theft From Motor Vehicle                        8896
Drug/Narcotic Violations                        7037
Burglary/Breaking And Entering                  6652
All Other Larceny                               6098
Alarm Business                                  5121
Aggravated Assault                              4736
Motor Vehicle Theft                             4726
Theft From Buildings                            4710
All Other Offenses                              4379
Drug Equipment Violations                       3853
Alarm Residental                                3542
Theft Of Motor Vehicle Parts Or Accessories     2634
False Pretense/Swindle/Confidence Game          2408
Property Lost                                   1855
Fraud Credit Card/Automated Teller Machines   

In [11]:
#Check for duplicate incidents
df.Case_Number.duplicated().value_counts()

False    226167
True      15171
Name: Case_Number, dtype: int64

In [12]:
df['Incident_Tract'].isnull().value_counts()

False    240025
True       1313
Name: Incident_Tract, dtype: int64

In [13]:
#Look at "Baker Zone" incidents, E1-E4, F5-F9
df[df['Incident_Tract'] =='E1']

,Address,City,State,Zip,Date_Incident,Jurisdiction,Incident_Tract,UCR_Incident_Code,Incident_Description,Incident_Type,Case_Number,Case_Status,Case_Status_Description,Latitude,Longitude,Location
3,3400 VINEWOOD DR,Chattanooga,TN,37406--,02/02/2015 11:38:00 PM,TN0330100,E1,000,Misc Report,Other Incidents,15-010660,NaN,NaN,35.074546,-85.239625,"(35.074546, -85.239625)"
671,3200 Amnicola HWY,Chattanooga,TN,37406,01/29/2016 04:35:00 PM,TN0330100,E1,35A,Drug/Narcotic Violations,Part 1 Crimes,16-009597,4,Inactive/Suspended,35.079473,-85.265261,"(35.079473, -85.265261)"
834,"3300 CAMPBELL ST,",Chattanooga,TN,37406-,06/05/2015 01:46:00 AM,TN0330100,E1,ALR,Alarm Residental,Other Incidents,15-053954,NaN,NaN,35.071740,-85.241089,"(35.07174, -85.241089)"
946,4200 BONNY OAKS DR,Chattanooga,TN,37416-,09/06/2015 12:00:00 PM,TN0330100,E1,000,Misc Report,Other Incidents,15-089686,NaN,NaN,35.080620,-85.217970,"(35.08062, -85.21797)"
1404,3400 Curtis St,Chattanooga,TN,37406-,05/01/2015 12:01:00 AM,TN0330100,E1,23D,Theft From Buildings,Part 1 Crimes,15-052747,3,Pending/Possible Leads,35.077516,-85.249416,"(35.077516, -85.249416)"
1676,300 Lookout High ST,Chattanooga,TN,37406,01/27/2017 05:00:00 PM,TN0330100,E1,13B,Simple Assault,Part 1 Crimes,17-008755,4,Inactive/Suspended,35.034856,-85.364183,"(35.034856, -85.364183)"
2234,3400 AMNICOLA HWY,CHATTANOOGA,TN,37406-,11/03/2015 10:01:00 PM,TN0330100,E1,000,Misc Report,Other Incidents,15-111003,4,Inactive/Suspended,35.081492,-85.263776,"(35.081492, -85.263776)"
2275,4300 AMNICOLA Hwy,Chattanooga,TN,37406-,05/20/2015 02:40:00 AM,TN0330100,E1,000,Misc Report,Other Incidents,15-047872,NaN,NaN,35.095980,-85.246034,"(35.09598, -85.246034)"
2974,2100 ALLIN ST.,Chattanooga,TN,37406-,11/07/2015 04:17:00 AM,TN0330100,E1,13B,Simple Assault,Part 1 Crimes,15-112193,2,Cleared by Arrest,35.076653,-85.249287,"(35.076653, -85.249287)"
3418,"3100 RIVERSIDE DR, CHATTANOOGA (WILDER ST/ELME...",Chattanooga,TN,37406-,08/31/2015 08:49:00 AM,TN0330100,E1,000,Misc Report,Other Incidents,15-087292,NaN,NaN,NaN,NaN,NaN


In [14]:
#Sort Previous Baker Zone incidents by Date_Incident
df[df['Incident_Tract'] =='E1'].sort_values(by=['Date_Incident'])

,Address,City,State,Zip,Date_Incident,Jurisdiction,Incident_Tract,UCR_Incident_Code,Incident_Description,Incident_Type,Case_Number,Case_Status,Case_Status_Description,Latitude,Longitude,Location
181067,"3300 TAYLOR ST, CHATTANOOGA (SEARLE ST/ALLIN S...",Chattanooga,TN,37406-,01/01/2015 01:31:00 PM,TN0330100,E1,000,Misc Report,Other Incidents,15-000195,NaN,NaN,NaN,NaN,NaN
180408,"2400 WATERHAVEN DR, CHATTANOOGA (SAFEHAVEN CT/...",Chattanooga,TN,37406-,01/01/2015 09:30:00 PM,TN0330100,E1,000,Misc Report,Other Incidents,15-000309,NaN,NaN,NaN,NaN,NaN
94792,3300 Campbell St.,Chattanooga,TN,37411-,01/01/2016 07:00:00 AM,TN0330100,E1,23D,Theft From Buildings,Part 1 Crimes,16-000147,NaN,NaN,35.072892,-85.240641,"(35.072892, -85.240641)"
109616,2100 Allin St,Chattanooga,TN,37406-,01/01/2016 12:11:00 AM,TN0330100,E1,000,Misc Report,Other Incidents,16-000006,NaN,NaN,35.077146,-85.249602,"(35.077146, -85.249602)"
165980,2100 Allin St,Chattanooga,TN,37406-,01/01/2016 12:16:00 AM,TN0330100,E1,520,Weapon Law Violations,Part 1 Crimes,16-000032,2,Cleared by Arrest,35.077002,-85.250149,"(35.077002, -85.250149)"
180723,"3400 AMNICOLA HWY, CHATTANOOGA (WISDOM ST/JUDD...",Chattanooga,TN,37406-,01/02/2015 06:10:00 PM,TN0330100,E1,CVM,Civil Matter,Other Incidents,15-000579,NaN,NaN,NaN,NaN,NaN
161379,"2500 N WILDER ST, CHATTANOOGA (WALNUT ST/PINE ...",Chattanooga,TN,37406-,01/02/2015 12:33:00 PM,TN0330100,E1,000,Misc Report,Other Incidents,15-000469,NaN,NaN,NaN,NaN,NaN
54030,"3700 GRACE AVE, CHATTANOOGA (DODSON AVE/MEADOW...",Chattanooga,TN,-,01/03/2015 01:04:00 AM,TN0330100,E1,000,Misc Report,Other Incidents,15-000708,NaN,NaN,NaN,NaN,NaN
181646,"4100 HAVEN ACRES LN, CHATTANOOGA (DEAD END/HAR...",Chattanooga,TN,37406-,01/03/2015 02:12:00 AM,TN0330100,E1,000,Misc Report,Other Incidents,15-000727,NaN,NaN,NaN,NaN,NaN
122232,"3100 NOA ST, CHATTANOOGA (WILDER ST/DEAD END) ...",Chattanooga,TN,37406-,01/03/2015 04:33:00 AM,TN0330100,E1,000,Misc Report,Other Incidents,15-000743,NaN,NaN,NaN,NaN,NaN


In [15]:
df[df['Incident_Tract'] =='B1'].sort_values(by=['Date_Incident'])

,Address,City,State,Zip,Date_Incident,Jurisdiction,Incident_Tract,UCR_Incident_Code,Incident_Description,Incident_Type,Case_Number,Case_Status,Case_Status_Description,Latitude,Longitude,Location
64802,3300 Dodson Ave,Chattanooga,TN,37406,01/01/2018 05:30:00 AM,TN0330100,B1,91Z,NaN,NaN,18-000087,3,Pending/Possible Leads,35.074855,-85.247780,"(35.074855099, -85.247779915)"
194116,2600 Boyce St,Chattanooga,TN,37406,01/01/2019 06:57:00 PM,TN0330100,B1,91Z,NaN,NaN,19-000295,3,Pending/Possible Leads,NaN,NaN,NaN
195562,4100 Haven Acres Ln,Chattanooga,TN,37406,01/01/2019 09:30:00 AM,TN0330100,B1,91Z,NaN,NaN,19-001595,3,Pending/Possible Leads,NaN,NaN,NaN
193828,3600 Faith Rd,Chattanooga,TN,37406,01/01/2019 12:00:00 AM,TN0330100,B1,91Z,NaN,NaN,19-000007,3,Pending/Possible Leads,NaN,NaN,NaN
51877,3200 Amnicola HWY,Chattanooga,TN,37406,01/02/2017 02:24:00 PM,TN0330100,B1,35A,Drug/Narcotic Violations,Part 1 Crimes,17-000490,4,Inactive/Suspended,35.079279,-85.264670,"(35.079279, -85.26467)"
9444,3400 Amnicola Hwy,Chattanooga,TN,37406-,01/02/2017 12:57:00 PM,TN0330100,B1,26A,False Pretense/Swindle/Confidence Game,Part 1 Crimes,17-002665,4,Inactive/Suspended,35.081383,-85.263115,"(35.081383, -85.263115)"
58387,3500 Marport Dr,Chattanooga,TN,37406,01/02/2018 02:30:00 PM,TN0330100,B1,26C,Impersonation,Part 1 Crimes,18-000455,3,Pending/Possible Leads,35.072862,-85.234296,"(35.072861726, -85.234296465)"
57840,1100 Judd Rd,Chattanooga,TN,37406,01/02/2018 05:00:00 AM,TN0330100,B1,91Z,NaN,NaN,18-000370,3,Pending/Possible Leads,35.083937,-85.264066,"(35.083937068, -85.264066163)"
63604,3500 Dodson Ave,Chattanooga,TN,37406,01/02/2018 07:00:00 AM,TN0330100,B1,91Z,NaN,NaN,18-000336,3,Pending/Possible Leads,35.077526,-85.247040,"(35.077525555, -85.247040296)"
61961,3400 Amnicola Hwy,Chattanooga,TN,37406,01/02/2018 10:04:00 PM,TN0330100,B1,35A,Drug/Narcotic Violations,Part 1 Crimes,18-000565,3,Pending/Possible Leads,35.082013,-85.264738,"(35.08201345, -85.264737525)"


<a id='cleaning'></a>
### Cleaning

- Make Column Names all lowercase 
- Incident Tract - 1313 null values
- Date_Incident split into date and time.  Do YYYY, MM, DD in order to sort by days
    - Change from string to int or date type
- Incident Tracts that are in Chattanooga Only - ER, RB, HC, etc are they included?
- Zip Code 
    - Drop "-" on the end, normalize into 5 digits
- Normalize Data
    - Date/Time
    -
- City
    - Spelling Errors
        - Locate all cities that start with "CHA" (loop)
        - If those cities = Chattanooga w/spelling errors, then change to Chattanooga
        - Check cities like Hixson, Ooltewah, etc that are within CPD jurisdiciton and change to Chattanooga
        - Drop Cities that are outside of CPD jurisdiction unless its cleared by arrest (loop)
    - Incidents Only Occuring within Chattanooga
- Drop "Jurisdiction" column, won't use in analysis - only 1 value
- Instead of cleaning the zones, clean and map via zipcode.  Check the zipcodes and clean for analysis. Drop zips outside of the Chattanooga area.  Check to see if zips are correct. 
- Mappable DataFrame - make a df that I can map 
    - 'map_CPD' df, drop rows with NaN values in location
    - Save to separate CSV to use in Tableau

In [16]:
#Make Copy of DF for cleaning
df_c = df.copy()

### Define 

Make all Column Names lowercase

### Code

In [17]:
df_c.columns = df_c.columns.str.lower()

### Test

In [18]:
df_c.columns

Index(['address', 'city', 'state', 'zip', 'date_incident', 'jurisdiction',
       'incident_tract', 'ucr_incident_code', 'incident_description',
       'incident_type', 'case_number', 'case_status',
       'case_status_description', 'latitude', 'longitude', 'location'],
      dtype='object')

### Define
Drop Column 'Jurisdiction' - Only one value, CPD

### Code

In [19]:
df_c.drop('jurisdiction', axis = 1, inplace = True)

In [20]:
#Test to make sure the column was dropped
df_c.head()

,address,city,state,zip,date_incident,incident_tract,ucr_incident_code,incident_description,incident_type,case_number,case_status,case_status_description,latitude,longitude,location
0,2200 Hamilton Place Blvd,Chattanooga,TN,37421,08/29/2017 11:20:00 AM,C7,23C,Shoplifting,Part 1 Crimes,17-084827,3,Pending/Possible Leads,35.039741,-85.155184,"(35.039741454, -85.155184058)"
1,2500 Mccrae St,Chattanooga,TN,37406,07/29/2017 11:01:00 PM,B2,91Z,NaN,NaN,17-073656,3,Pending/Possible Leads,35.058561,-85.252378,"(35.058561443, -85.252378156)"
2,1100 THOMAS LN.,Chattanooga,TN,37343-,11/08/2015 10:00:00 AM,A1,220,Burglary/Breaking And Entering,Part 1 Crimes,15-112814,3,Pending/Possible Leads,35.142962,-85.247589,"(35.142962, -85.247589)"
3,3400 VINEWOOD DR,Chattanooga,TN,37406--,02/02/2015 11:38:00 PM,E1,000,Misc Report,Other Incidents,15-010660,NaN,NaN,35.074546,-85.239625,"(35.074546, -85.239625)"
4,2800 Dodson Ave,Chattanooga,TN,37415-,04/08/2015 12:18:00 PM,E2,000,Misc Report,Other Incidents,15-032003,NaN,NaN,35.069329,-85.250511,"(35.069329, -85.250511)"


### Define

Normalize Zip code into 5 digit.

### Code

In [21]:
df_c.zip = df_c.zip.str[:5]

### Test

In [22]:
df_c.zip.head()

0    37421
1    37406
2    37343
3    37406
4    37415
Name: zip, dtype: object

### Define

date_incident - split into date and time into different columns. 

date_incident & time_incident as columns names

drop date_incident then change incident_date into date_incident

time_incident change into 24 hr time from AM/PM

date_incident change into YYYY/MM/DD for analysis

### Code

In [23]:
#Cut time portion into new column
df_c['time_incident'] = df_c.date_incident.str[10:]

In [24]:
#Cut date portion into new column
df_c['incident_date'] = df_c.date_incident.str[:10]

In [25]:
#Drop original Column
df_c.drop('date_incident', axis = 1, inplace = True)

In [26]:
#Rename incident_date column back into date_incident
df_c = df_c.rename(columns={'incident_date' : 'date_incident'})

### Test

In [27]:
df_c.head()

,address,city,state,zip,incident_tract,ucr_incident_code,incident_description,incident_type,case_number,case_status,case_status_description,latitude,longitude,location,time_incident,date_incident
0,2200 Hamilton Place Blvd,Chattanooga,TN,37421,C7,23C,Shoplifting,Part 1 Crimes,17-084827,3,Pending/Possible Leads,35.039741,-85.155184,"(35.039741454, -85.155184058)",11:20:00 AM,08/29/2017
1,2500 Mccrae St,Chattanooga,TN,37406,B2,91Z,NaN,NaN,17-073656,3,Pending/Possible Leads,35.058561,-85.252378,"(35.058561443, -85.252378156)",11:01:00 PM,07/29/2017
2,1100 THOMAS LN.,Chattanooga,TN,37343,A1,220,Burglary/Breaking And Entering,Part 1 Crimes,15-112814,3,Pending/Possible Leads,35.142962,-85.247589,"(35.142962, -85.247589)",10:00:00 AM,11/08/2015
3,3400 VINEWOOD DR,Chattanooga,TN,37406,E1,000,Misc Report,Other Incidents,15-010660,NaN,NaN,35.074546,-85.239625,"(35.074546, -85.239625)",11:38:00 PM,02/02/2015
4,2800 Dodson Ave,Chattanooga,TN,37415,E2,000,Misc Report,Other Incidents,15-032003,NaN,NaN,35.069329,-85.250511,"(35.069329, -85.250511)",12:18:00 PM,04/08/2015


### Define

- Locate null values in 'city' column

- Replace null values in 'city' column with Chattanooga

In [28]:
### Look at city values that are null
df_c['city'].isnull().value_counts()

False    241243
True         95
Name: city, dtype: int64

In [29]:
df_c[pd.isnull(df_c.city)]

,address,city,state,zip,incident_tract,ucr_incident_code,incident_description,incident_type,case_number,case_status,case_status_description,latitude,longitude,location,time_incident,date_incident
5362,6200 SHALLOWFORD RD,NaN,TN,NaN,G6,000,Misc Report,Other Incidents,16-010094,NaN,NaN,35.053291,-85.178338,"(35.053291, -85.178338)",04:18:00 AM,01/31/2016
5803,700 E 11th St,NaN,TN,37403,A9,000,Misc Report,Other Incidents,16-017196,NaN,NaN,35.040921,-85.299147,"(35.040921, -85.299147)",09:42:00 AM,02/20/2016
20958,7000 McCutcheon RD,NaN,TN,NaN,C9,000,Misc Report,Other Incidents,16-020081,NaN,NaN,35.046613,-85.156853,"(35.046613, -85.156853)",11:00:00 AM,02/28/2016
26333,3000 WOODSIDE ST,NaN,TN,NaN,F8,000,Misc Report,Other Incidents,16-000772,NaN,NaN,35.011335,-85.273301,"(35.011335, -85.273301)",12:27:00 PM,01/03/2016
37579,2100 E 4th ST,NaN,TN,NaN,E3,RVO,Vehicle Recovered Stolen Outside Of Chattanooga,Other Incidents,15-117121,2,Cleared by Arrest,35.038509,-85.270468,"(35.038509, -85.270468)",11:17:00 PM,11/20/2015
38923,100 E 20TH ST,NaN,TN,NaN,A10,000,Misc Report,Other Incidents,16-015708,NaN,NaN,35.031413,-85.307560,"(35.031413, -85.30756)",11:55:00 AM,02/16/2016
50691,6800 Tyner RD,NaN,TN,37421,C9,000,Misc Report,Other Incidents,16-015770,NaN,NaN,35.062249,-85.156158,"(35.062249, -85.156158)",01:33:00 PM,02/16/2016
69698,1500 HICKORY VALLEY RD,NaN,TN,NaN,C4,000,Misc Report,Other Incidents,16-020108,NaN,NaN,35.022511,-85.178834,"(35.022511, -85.178834)",08:00:00 PM,02/27/2016
77706,MARKET ST,NaN,TN,NaN,B7,000,Misc Report,Other Incidents,16-003487,NaN,NaN,35.056979,-85.310137,"(35.056979, -85.310137)",09:00:00 AM,01/11/2016
78388,7000 SHALLOWFORD RD,NaN,TN,NaN,G6,000,Misc Report,Other Incidents,16-005771,NaN,NaN,35.043561,-85.157668,"(35.043561, -85.157668)",11:20:00 PM,01/17/2016


### Code

In [30]:
#need to fill null valaues in cities to be able to fix spelling errors
values_city = {'city': 'Chattanooga'}
df_c.fillna(value = values_city, inplace = True)

### Test

In [31]:
df_c['city'].isnull().value_counts()

False    241338
Name: city, dtype: int64

### Define

- Find spelling errors in 'city' column

- Change all spelling errors that are related to Chattanooga to Chattanooga for analysis purposes

In [32]:
df_c.city.value_counts()

Chattanooga        211529
CHATTANOOGA         25147
Hixson               3137
HIXSON                301
chattanooga           240
Ooltewah              148
CHATT                  72
Chattanoga             61
OOLTEWAH               40
CHATTANOGOA            39
Chattanooga,           32
Chattanoogaq           31
CH                     28
CHATANOOGA             27
Lupton City            26
CHATTANOOG             23
East Ridge             21
RED BANK               15
Chatanooga             15
CHATTNAOOGA            14
Chatt                  13
Chattannoga            13
hixson                 11
Chatttanooga           11
EAST RIDGE             11
CHATTANOOA             11
CHATTTANOOGA           11
CHATTANOGA             10
Harrison               10
Chattnooga              9
                    ...  
Chattnaooga             1
SIGNAL MTN.             1
CHATTANOOGSA            1
RINGGOLD                1
Spencer                 1
Chattano0ga             1
CHATTANOOGA,,,,         1
NORWICH     

In [33]:
#Change misspelled city names to Chattanooga
df_c.replace({'city':{'CHATTANOOGA' : 'Chattanooga', 'Hixson':'Chattanooga'}}, inplace = True)

In [34]:
#Continue to replace misspelled city names to Chattanooga, work in 5 or 6 at a time to test each time.
df_c.replace({'city':{'HIXSON' : 'Chattanooga', 'chattanooga':'Chattanooga', 'Ooltewah':'Chattanooga', 'CHATT':'Chattanooga', 'Chattanoga':'Chattanooga'}}, inplace = True)

In [35]:
#Continue
df_c.replace({'city':{'OOLTEWAH' : 'Chattanooga', 'CHATTANOGOA':'Chattanooga', 'Chattanooga,':'Chattanooga', 'CH':'Chattanooga', 'Chattanoogaq':'Chattanooga'}}, inplace = True)

In [36]:
#Continue
df_c.replace({'city':{'CHATANOOGA' : 'Chattanooga', 'Lupton City':'Chattanooga', 'CHATTANOOG':'Chattanooga', 'CHATTNAOOGA':'Chattanooga', 'Chatanooga':'Chattanooga'}}, inplace = True)

In [37]:
#Continue
df_c.replace({'city':{'Chattannoga' : 'Chattanooga', 'Chatt':'Chattanooga', 'CHATTTANOOGA':'Chattanooga', 'Chatttanooga':'Chattanooga', 'hixson':'Chattanooga'}}, inplace = True)

In [38]:
#Continue
df_c.replace({'city':{'CHATTANOOA' : 'Chattanooga', 'CHATTANOGA':'Chattanooga', 'Chattnooga':'Chattanooga', 'CHattanooga':'Chattanooga', 'chattanogoa':'Chattanooga'}}, inplace = True)

In [39]:
#Continue
df_c.replace({'city':{'CAHTTANOOGA' : 'Chattanooga', 'CHATTANOOGA,':'Chattanooga', 'HIxson':'Chattanooga', 'Chattanogoa':'Chattanooga', 'chatt':'Chattanooga'}}, inplace = True)

In [40]:
#Continue
df_c.replace({'city':{'Chattanoooga' : 'Chattanooga', 'Chatt.':'Chattanooga', 'Chattanoog':'Chattanooga', 'CHATTA':'Chattanooga', 'CHATTANOOAG':'Chattanooga'}}, inplace = True)

In [41]:
#Continue
df_c.replace({'city':{'HIXSON PIKE' : 'Chattanooga', 'Chattaooga':'Chattanooga', '.Chattanooga':'Chattanooga', 'CHA':'Chattanooga', 'Chattanooag':'Chattanooga'}}, inplace = True)

In [42]:
#Continue
df_c.replace({'city':{'CHATTAANOOGA' : 'Chattanooga', 'ooltewah':'Chattanooga', 'chattannoga':'Chattanooga', 'Chattanoo':'Chattanooga', 'CHATTANOOOGA':'Chattanooga'}}, inplace = True)

In [43]:
#Continue
df_c.replace({'city':{'Chattanooa' : 'Chattanooga', 'CHATTANOGGA':'Chattanooga', 'cHATTANOOGA':'Chattanooga', 'CHATTANOOGA TN':'Chattanooga', 'CHATTNOOGA':'Chattanooga'}}, inplace = True)

In [44]:
#Continue
df_c.replace({'city':{'CHATTANO9OGA' : 'Chattanooga', 'LUPTON CITY':'Chattanooga', 'chattanooa':'Chattanooga', 'chatanooga':'Chattanooga', 'Chattanoogat':'Chattanooga'}}, inplace = True)

In [45]:
#Continue
df_c.replace({'city':{'Chattanogga' : 'Chattanooga', 'Chattanooga t':'Chattanooga', 'Chaatanooga':'Chattanooga', 'hIXSON':'Chattanooga', 'Chattanoonga':'Chattanooga'}}, inplace = True)

In [46]:
#Continue
df_c.replace({'city':{'CHATTA0OGA' : 'Chattanooga', 'TIFTONIA':'Chattanooga', 'ChattANOOGA':'Chattanooga', 'Chattanaooga':'Chattanooga', 'CHATTanooga':'Chattanooga'}}, inplace = True)

In [47]:
#Continue
df_c.replace({'city':{'Hixon' : 'Chattanooga', 'CHATTAN0OGA':'Chattanooga', 'ChattanoogA':'Chattanooga', 'Chaattanooga':'Chattanooga', 'CHATTANNOOGA':'Chattanooga'}}, inplace = True)

In [48]:
#Continue
df_c.replace({'city':{'RED BANK' : 'Red Bank', 'EAST RIDGE':'East Ridge', 'HARRISON':'Harrison', 'Hixon':'Chattanooga', 'CHATTANOOGA, TN':'Chattanooga'}}, inplace = True)

In [49]:
#Continue
df_c.replace({'city':{'HIXSON TN' : 'Chattanooga', 'Chattanoofa':'Chattanooga', 'Chattan ooga':'Chattanooga', 'CHATTANAOOGA':'Chattanooga', 'Hisxon':'Chattanooga'}}, inplace = True)

In [50]:
#Continue
df_c.replace({'city':{'Chattqanooga' : 'Chattanooga', 'CHATTANOOGA,,,,':'Chattanooga', 'Chatanoogat':'Chattanooga', 'CATTANOOGA':'Chattanooga', 'Chattagoona':'Chattanooga'}}, inplace = True)

In [51]:
#Continue
df_c.replace({'city':{'OOTHEWAH' : 'Chattanooga', 'CHATTAOOGA':'Chattanooga', 'CHattanoogq':'Chattanooga', 'Chattqnoogq':'Chattanooga', 'tn':'Chattanooga'}}, inplace = True)

In [52]:
#Continue
df_c.replace({'city':{'CHATTA NIOOGA' : 'Chattanooga', 'Chattanoogq':'Chattanooga', 'Chattanooga Tn':'Chattanooga', 'Tifftonia':'Chattanooga', 'CHATTANOOGSA':'Chattanooga'}}, inplace = True)

In [53]:
#Continue
df_c.replace({'city':{'CHHATTANOOGA' : 'Chattanooga', 'CHATTANNOGA':'Chattanooga', 'CHATATANOOGA':'Chattanooga', 'Chattano0ga':'Chattanooga', 'chatttanooga':'Chattanooga'}}, inplace = True)

In [54]:
#Continue
df_c.replace({'city':{'CHSTTSNOOGA' : 'Chattanooga', 'Chattnaooga':'Chattanooga', 'CHATTANOOGA,TN':'Chattanooga', 'ChattanoogaH ':'Chattanooga', 'CJHATTANOOGA':'Chattanooga'}}, inplace = True)

In [55]:
#Continue
df_c.replace({'city':{'CHATATNOOGA' : 'Chattanooga', 'Hixsin':'Chattanooga', 'CHATTANIOOGA':'Chattanooga', 'CHATTANIOOGA':'Chattanooga', 'chattaooga':'Chattanooga'}}, inplace = True)

In [56]:
#Continue
df_c.replace({'city':{'ChattanoogaH' : 'Chattanooga', 'COLLEGEDALE':'Collegedale', 'Cahattanooga':'Chattanooga', '"\"':'Chattanooga', 'Chattanooge':'Chattanooga'}}, inplace = True)

In [57]:
#Continue
df_c.replace({'city':{'CHATTANO0GA' :'Chattanooga', 'Hixson Pike':'Chattanooga', 'Chatanoogs':'Chattanooga', 'Chattanppga':'Chattanooga', 'Chattanooaga':'Chattanooga'}}, inplace = True)

In [58]:
#Continue
df_c.replace({'city':{'Chattanoogs' : 'Chattanooga', 'Ooltwah':'Chattanooga', 'Hixson,':'Chattanooga', 'Chattanooga, Tn':'Chattanooga', 'Ooletwah':'Chattanooga', 'CHATTANDOOGA' : 'Chattanooga', 'Red Bank':'Chattanooga'}}, inplace = True)

### Test

In [59]:
df_c.city.value_counts()

Chattanooga        241243
East Ridge             32
Harrison               15
Collegedale             5
Grueti-Laager           5
HAMILTON COUNTY         5
Wildwood                3
Unincorporated          2
Apison                  2
Jasper                  1
Chattanooha             1
\                       1
Dunlap                  1
37401                   1
Maryville               1
Fort Oglethorpe         1
Lake Charles            1
Clifton                 1
Dalton                  1
Soddy Daisy             1
RINGGOLD                1
Chattooga               1
Chattnoooga             1
NORWICH                 1
Spencer                 1
Gwinnett                1
BLVD                    1
Ft. Oglethorpe          1
hattanoga               1
D305                    1
Hayes                   1
Chattaanooga            1
37476                   1
SIGNAL MTN.             1
Chatta nooga            1
Name: city, dtype: int64

### Define

Check non-Chattanooga incidents within City column to see what the incidents are and if they can be dropped.

In [60]:
df_c[df_c['city'] != 'Chattanooga']

,address,city,state,zip,incident_tract,ucr_incident_code,incident_description,incident_type,case_number,case_status,case_status_description,latitude,longitude,location,time_incident,date_incident
887,9300 CRYSTAL BROOK DR,Apison,TN,37302,C1,23G,Theft Of Motor Vehicle Parts Or Accessories,Part 1 Crimes,17-008062,4,Inactive/Suspended,NaN,NaN,NaN,12:00:00 PM,01/20/2017
1108,00 W Patterson Ave,Ft. Oglethorpe,GA,30742,B7,13B,Simple Assault,Part 1 Crimes,16-069799,4,Inactive/Suspended,NaN,NaN,NaN,11:00:00 PM,07/04/2016
3828,"3300 GLEASON DR, EAST RIDGE (EILEEN LN/DARRYL ...",East Ridge,TN,37402,A6,000,Misc Report,Other Incidents,16-076649,NaN,NaN,NaN,NaN,NaN,07:57:00 AM,07/22/2016
4432,6100 Hwy 58,Harrison,TN,37341,NaN,26B,Fraud Credit Card/Automated Teller Machines,Part 1 Crimes,17-010738A,NaN,NaN,35.118565,-85.124300,"(35.118565, -85.1243)",06:14:00 PM,02/02/2017
9368,9200 LEE HWY,Collegedale,TN,37421,G9,000,Misc Report,Other Incidents,15-123454,NaN,NaN,35.084973,-85.062624,"(35.084973, -85.062624)",11:31:00 PM,12/08/2015
12426,5600 Highway 58.,Harrison,TN,37341,C5,23H,All Other Larceny,Part 1 Crimes,17-007939,3,Pending/Possible Leads,35.108233,-85.144699,"(35.108233, -85.144699)",05:00:00 AM,01/25/2017
14812,4000 KIRKLAND AVE,Harrison,TN,37410,A12,CVM,Civil Matter,Other Incidents,16-088870,4,Inactive/Suspended,35.004377,-85.314114,"(35.004377, -85.314114)",08:25:00 PM,08/23/2016
15814,4000 KIRKLAND AVE,Harrison,TN,37410,A12,23H,All Other Larceny,Part 1 Crimes,16-088870,4,Inactive/Suspended,35.004377,-85.314114,"(35.004377, -85.314114)",08:25:00 PM,08/23/2016
19776,4600 Northland LN,Harrison,TN,37416,G5,220,Burglary/Breaking And Entering,Part 1 Crimes,15-098427,3,Pending/Possible Leads,35.088267,-85.194695,"(35.088267, -85.194695)",07:37:00 PM,09/30/2015
20218,1200 Boynton Dr,Chattaanooga,TN,37402,A6,DON,Dead On Arrival Natural,Other Incidents,17-021264,NaN,NaN,35.045603,-85.319646,"(35.045603, -85.319646)",10:00:00 AM,03/07/2017


### Code

In [61]:
#Drop rows that do not have Chattanooga in the City
df_c = df_c[df_c['city'] == 'Chattanooga']

### Test

In [62]:
df_c.head()

,address,city,state,zip,incident_tract,ucr_incident_code,incident_description,incident_type,case_number,case_status,case_status_description,latitude,longitude,location,time_incident,date_incident
0,2200 Hamilton Place Blvd,Chattanooga,TN,37421,C7,23C,Shoplifting,Part 1 Crimes,17-084827,3,Pending/Possible Leads,35.039741,-85.155184,"(35.039741454, -85.155184058)",11:20:00 AM,08/29/2017
1,2500 Mccrae St,Chattanooga,TN,37406,B2,91Z,NaN,NaN,17-073656,3,Pending/Possible Leads,35.058561,-85.252378,"(35.058561443, -85.252378156)",11:01:00 PM,07/29/2017
2,1100 THOMAS LN.,Chattanooga,TN,37343,A1,220,Burglary/Breaking And Entering,Part 1 Crimes,15-112814,3,Pending/Possible Leads,35.142962,-85.247589,"(35.142962, -85.247589)",10:00:00 AM,11/08/2015
3,3400 VINEWOOD DR,Chattanooga,TN,37406,E1,000,Misc Report,Other Incidents,15-010660,NaN,NaN,35.074546,-85.239625,"(35.074546, -85.239625)",11:38:00 PM,02/02/2015
4,2800 Dodson Ave,Chattanooga,TN,37415,E2,000,Misc Report,Other Incidents,15-032003,NaN,NaN,35.069329,-85.250511,"(35.069329, -85.250511)",12:18:00 PM,04/08/2015


In [63]:
df_c['city'].value_counts()

Chattanooga    241243
Name: city, dtype: int64

### Define

Clean incident_type into three columns: Part 1 Crimes, Part 2 Crimes, Other Incidents with values "yes" and "no"

### Code

In [64]:
#Use numpy's 'where' function to locate the three values within 
df_c['part_1_crime'] = np.where(df_c['incident_type'] == 'Part 1 Crimes', 'yes', 'no')
df_c['part_2_crime'] = np.where(df_c['incident_type'] == 'Part 2 Crimes', 'yes', 'no')
df_c['other_incidents'] = np.where(df_c['incident_type'] == 'Other Incidents', 'yes', 'no')

### Test

In [65]:
df_c.head()

,address,city,state,zip,incident_tract,ucr_incident_code,incident_description,incident_type,case_number,case_status,case_status_description,latitude,longitude,location,time_incident,date_incident,part_1_crime,part_2_crime,other_incidents
0,2200 Hamilton Place Blvd,Chattanooga,TN,37421,C7,23C,Shoplifting,Part 1 Crimes,17-084827,3,Pending/Possible Leads,35.039741,-85.155184,"(35.039741454, -85.155184058)",11:20:00 AM,08/29/2017,yes,no,no
1,2500 Mccrae St,Chattanooga,TN,37406,B2,91Z,NaN,NaN,17-073656,3,Pending/Possible Leads,35.058561,-85.252378,"(35.058561443, -85.252378156)",11:01:00 PM,07/29/2017,no,no,no
2,1100 THOMAS LN.,Chattanooga,TN,37343,A1,220,Burglary/Breaking And Entering,Part 1 Crimes,15-112814,3,Pending/Possible Leads,35.142962,-85.247589,"(35.142962, -85.247589)",10:00:00 AM,11/08/2015,yes,no,no
3,3400 VINEWOOD DR,Chattanooga,TN,37406,E1,000,Misc Report,Other Incidents,15-010660,NaN,NaN,35.074546,-85.239625,"(35.074546, -85.239625)",11:38:00 PM,02/02/2015,no,no,yes
4,2800 Dodson Ave,Chattanooga,TN,37415,E2,000,Misc Report,Other Incidents,15-032003,NaN,NaN,35.069329,-85.250511,"(35.069329, -85.250511)",12:18:00 PM,04/08/2015,no,no,yes


In [66]:
df_c['incident_type'].value_counts()

Part 1 Crimes      94143
Other Incidents    77913
Part 2 Crimes      10303
Name: incident_type, dtype: int64

In [67]:
### Drop incident_type column
df_c.drop('incident_type', axis =1, inplace = True)

In [68]:
df_c.head()

,address,city,state,zip,incident_tract,ucr_incident_code,incident_description,case_number,case_status,case_status_description,latitude,longitude,location,time_incident,date_incident,part_1_crime,part_2_crime,other_incidents
0,2200 Hamilton Place Blvd,Chattanooga,TN,37421,C7,23C,Shoplifting,17-084827,3,Pending/Possible Leads,35.039741,-85.155184,"(35.039741454, -85.155184058)",11:20:00 AM,08/29/2017,yes,no,no
1,2500 Mccrae St,Chattanooga,TN,37406,B2,91Z,NaN,17-073656,3,Pending/Possible Leads,35.058561,-85.252378,"(35.058561443, -85.252378156)",11:01:00 PM,07/29/2017,no,no,no
2,1100 THOMAS LN.,Chattanooga,TN,37343,A1,220,Burglary/Breaking And Entering,15-112814,3,Pending/Possible Leads,35.142962,-85.247589,"(35.142962, -85.247589)",10:00:00 AM,11/08/2015,yes,no,no
3,3400 VINEWOOD DR,Chattanooga,TN,37406,E1,000,Misc Report,15-010660,NaN,NaN,35.074546,-85.239625,"(35.074546, -85.239625)",11:38:00 PM,02/02/2015,no,no,yes
4,2800 Dodson Ave,Chattanooga,TN,37415,E2,000,Misc Report,15-032003,NaN,NaN,35.069329,-85.250511,"(35.069329, -85.250511)",12:18:00 PM,04/08/2015,no,no,yes


### Define

- Clean zipcodes to make sure all zip codes are within Chattanooga
- Check to make sure zip codes are correct, numbers, etc


In [69]:
df_c['zip'].value_counts()

37421    46096
37411    27500
37406    23057
37404    22522
37343    22388
37407    17918
37402    14821
37405    12680
37416    11262
37410     8867
37415     8198
37408     6893
37419     6504
37403     5997
37409     2996
37363     1305
-          509
37412      505
37351      223
37341      200
37401       80
37377       36
37450       23
37311       22
37434       20
37315       18
-3740       16
37413       16
37321       13
37367       13
         ...  
47405        1
3748-        1
34919        1
34702        1
27405        1
30741        1
7406-        1
30408        1
38411        1
47419        1
71976        1
34708        1
-374         1
62436        1
37319        1
37444        1
37521        1
17404        1
34432        1
63741        1
3702         1
37241        1
36419        1
38415        1
37           1
37605        1
374-2        1
3742         1
47306        1
33740        1
Name: zip, Length: 251, dtype: int64

##### Zip Code Cleaning
Get list of zips that are not complete or are miss spelled. This will take some time to figure out where each actual zip is located using each address. Using the list above find change each zip one by one.  This cleaning would not be necessary if the zip codes were input correctly.  We can't drop the wrong zips becuase we would loose incidents that we want to analyse. Similar to the above coding for each city this will take several lines of code to fix.  We will change 5-6 zipcodes at a time per line of code.



In [70]:
df_c['zip'].value_counts()

37421    46096
37411    27500
37406    23057
37404    22522
37343    22388
37407    17918
37402    14821
37405    12680
37416    11262
37410     8867
37415     8198
37408     6893
37419     6504
37403     5997
37409     2996
37363     1305
-          509
37412      505
37351      223
37341      200
37401       80
37377       36
37450       23
37311       22
37434       20
37315       18
-3740       16
37413       16
37321       13
37367       13
         ...  
47405        1
3748-        1
34919        1
34702        1
27405        1
30741        1
7406-        1
30408        1
38411        1
47419        1
71976        1
34708        1
-374         1
62436        1
37319        1
37444        1
37521        1
17404        1
34432        1
63741        1
3702         1
37241        1
36419        1
38415        1
37           1
37605        1
374-2        1
3742         1
47306        1
33740        1
Name: zip, Length: 251, dtype: int64

In [71]:
df_c.replace({'zip':{'32410' : '37410', '3710':'37410', '3711':'37411', '37490':'37409', '38411':'37411', '47419':'37419', '37521':'37421'}}, inplace = True)

In [72]:
#Continue cleaning
df_c.replace({'zip':{'37434' : '37343', '24522':'37408', '37334':'37343', '3742':'37421', '62327':'37421', '37430':'37407'}}, inplace = True)

In [73]:
df_c.replace({'zip':{'374' : '37405', '36411':'37411', '-3640':'37404', '11320':'37421', '3742':'37421', '30408':'37408'}}, inplace = True)

In [74]:
df_c.replace({'zip':{'39573' : '37421', '73411':'37411', '-3640':'37406', '3742':'37421', '37043':'37403', '37241':'37421'}}, inplace = True)

In [75]:
df_c.replace({'zip':{'3-740' : '37408', '63741':'37411', '37423':'37343', '3742 ':'37402', '47403':'37403', '37342':'37421'}}, inplace = True)

In [76]:
df_c.replace({'zip':{'37-40' : '37402', '207':'37411', '37444':'37405', '36343':'37343', '34419':'37419', '704-':'37404'}}, inplace = True)

In [77]:
df_c.replace({'zip':{'37426' : '37419', '37348':'37343', '4711-':'37411', '27410':'37410', '34416':'37416', '37019':'37419'}}, inplace = True)

In [78]:
df_c.replace({'zip':{'37431' : '37416', '374-0':'37408', '36404':'37404', '7343':'37343', '36419':'37419', '3411':'37411'}}, inplace = True)

In [79]:
df_c.replace({'zip':{'37' : '37421', '37102':'37421', '37354':'37351', '73421':'37421', '62436':'37421', '34708':'37408'}}, inplace = True)

In [80]:
df_c.replace({'zip':{'37309' : '37408', '3702':'37402', '37503':'37408', '38421':'37421', '37466':'37406', '62881':'37421'}}, inplace = True)

In [81]:
df_c.replace({'zip':{'3702-' : '3702', '98421':'37421', '34373':'37343', '31416':'37416', '36415':'37415', '27363':'37363'}}, inplace = True)

In [82]:
df_c.replace({'zip':{'37040' : '37404', '47405':'37405', '7406-':'37406', '37473':'37343', '30411':'37411', '37319':'37419'}}, inplace = True)

In [83]:
df_c.replace({'zip':{'3407-' : '37407', '3747-':'37404', '4711 ':'37411', '39411':'37411', '34416':'37416', '374-2':'37402'}}, inplace = True)

In [84]:
df_c.replace({'zip':{'-374' : '37403', '84878':'37415', '3745-':'37405', '37741':'37411', '34919':'37419', '73402':'37402'}}, inplace = True)

In [85]:
df_c.replace({'zip':{'3748-' : '37409', '38416':'37416', '34404':'37404', '37443':'37343', '3719-':'37419', '37423':'37419'}}, inplace = True)

In [86]:
df_c.replace({'zip':{'37605' : '37406', '17410':'37410', '17404':'37404', '27410':'37410', '3343-':'37343', '27415':'37415'}}, inplace = True)

In [87]:
df_c.replace({'zip':{'37460' : '37406', '-3744':'37419', '37400':'37415', '71976':'37421', '34740':'37404', '33740':'37404'}}, inplace = True)

In [88]:
df_c.replace({'zip':{'37420' : '37419', '34702':'37408', '47505':'37405', '27416':'37406', '410':'37410', '3706-':'37406'}}, inplace = True)

In [89]:
df_c.replace({'zip':{'3711-' : '37411', '7405-':'37415', '21416':'37416', '3702':'37402', '37478':'37343', '57421':'37421'}}, inplace = True)

In [90]:
df_c.replace({'zip':{'37046' : '37406', '37604':'37406', '3721-':'37421', '36412':'37410', '47415':'37415', '3716-':'37416'}}, inplace = True)

In [91]:
df_c.replace({'zip':{'4711' : '37411', '30741':'37402', '3734-':'37343', '34432':'37421', '34416':'37416', '33734':'37343'}}, inplace = True)

In [92]:
df_c.replace({'zip':{'47410' : '37410', '37496':'37408', '37376':'37363', '39316':'37416', '37310':'37410', '47306':'37411'}}, inplace = True)

In [93]:
df_c.replace({'zip':{'34706':'37406', '37346':'37343', '3604':'37406', '373-4':'37343', '38415':'37415', '39410':'37410'}}, inplace = True)

In [94]:
df_c.replace({'zip':{'37308' : '37415', '3742-':'37421', '3707-':'37407', '37432':'37419', '37140':'37410', '36416':'37407'}}, inplace = True)

In [95]:
df_c.replace({'zip':{'36405' : '37403', '27405':'37405', '31406':'37406', '37211':'37411', '34416':'37416', '3404-':'37404'}}, inplace = True)

In [106]:
df_c.replace({'zip':{'37042' : '37402', '37348':'37343', '4711-':'37411', '27410':'37410', '34416':'37416', '37019':'37419', '30831':'37363'}}, inplace = True)

In [107]:
df_c.replace({'zip':{'47343': '37343', '36403':'37403', '37505':'37405','3421':'37421',}}, inplace = True)

In [112]:
#check to see where individual zipcodes are located. 
#The zipcodes that are known to not be in the Chattanooga area are the ones changed above
df_c[df_c['zip'] == '38407']

,address,city,state,zip,incident_tract,ucr_incident_code,incident_description,case_number,case_status,case_status_description,latitude,longitude,location,time_incident,date_incident,part_1_crime,part_2_crime,other_incidents
5423,E 11th st,Chattanooga,TN,38407,A9,000,Misc Report,17-014621,NaN,NaN,35.043481,-85.310122,"(35.043481, -85.310122)",12:33:00 AM,02/15/2017,no,no,yes
47363,3000 4th Ave.,Chattanooga,TN,38407,B8,000,Misc Report,16-133327,NaN,NaN,35.009329,-85.284203,"(35.009329, -85.284203)",02:00:00 AM,12/29/2016,no,no,yes


In [108]:
df_c['zip'].value_counts()

37421    46118
37411    27516
37406    23071
37404    22533
37343    22424
37407    17923
37402    14829
37405    12688
37416    11269
37410     8877
37415     8207
37408     6903
37419     6515
37403     6003
37409     2998
37363     1310
-          509
37412      505
37351      224
37341      200
37401       80
37377       36
37450       23
37311       22
37315       18
-3740       16
37413       16
37321       13
37367       13
37379       12
         ...  
37422        3
34343        2
3741         2
37307        2
47416        2
30721        2
37417        2
37515        2
30742        2
73406        2
37418        2
30407        2
37497        2
37302        2
38410        2
-3734        2
3721         2
37347        2
27409        2
37350        2
37323        2
37514        2
37146        2
38405        2
34704        2
37047        2
37306        2
73404        2
37121        2
38407        2
Name: zip, Length: 105, dtype: int64

At this point I am going to halt cleaning the zipcodes becuase there are an endless number of mistakes.  I will analyse the data and see how many outliers or problems arise due to the zipcode issue.  I may return to this later using google sheets to clean individual zips as needed

The next step will be to set the case_number as the index. Case numbers are generated in numerical order. The first two digits before the '-' signify the year. CPD uses 6 digits after the '-'. The first case number in the year 2015 would be '15-000001'.

In [110]:
df_c.set_index(['case_number'])

,address,city,state,zip,incident_tract,ucr_incident_code,incident_description,case_status,case_status_description,latitude,longitude,location,time_incident,date_incident,part_1_crime,part_2_crime,other_incidents
case_number,,,,,,,,,,,,,,,,,
17-084827,2200 Hamilton Place Blvd,Chattanooga,TN,37421,C7,23C,Shoplifting,3,Pending/Possible Leads,35.039741,-85.155184,"(35.039741454, -85.155184058)",11:20:00 AM,08/29/2017,yes,no,no
17-073656,2500 Mccrae St,Chattanooga,TN,37406,B2,91Z,NaN,3,Pending/Possible Leads,35.058561,-85.252378,"(35.058561443, -85.252378156)",11:01:00 PM,07/29/2017,no,no,no
15-112814,1100 THOMAS LN.,Chattanooga,TN,37343,A1,220,Burglary/Breaking And Entering,3,Pending/Possible Leads,35.142962,-85.247589,"(35.142962, -85.247589)",10:00:00 AM,11/08/2015,yes,no,no
15-010660,3400 VINEWOOD DR,Chattanooga,TN,37406,E1,000,Misc Report,NaN,NaN,35.074546,-85.239625,"(35.074546, -85.239625)",11:38:00 PM,02/02/2015,no,no,yes
15-032003,2800 Dodson Ave,Chattanooga,TN,37415,E2,000,Misc Report,NaN,NaN,35.069329,-85.250511,"(35.069329, -85.250511)",12:18:00 PM,04/08/2015,no,no,yes
17-010334,5900 Lee Hwy,Chattanooga,TN,37411,C4,290,Destructive/Damage/Vandalism Of Property,1,Unfounded,NaN,NaN,NaN,02:50:00 PM,02/01/2017,yes,no,no
17-006791,2200 E12th ST,Chattanooga,TN,37404,B5,520,Weapon Law Violations,2,Cleared by Arrest,NaN,NaN,NaN,04:34:00 PM,01/21/2017,yes,no,no
17-007591,2400 Long ST,Chattanooga,TN,37409,A10,13A,Aggravated Assault,2,Cleared by Arrest,35.028397,-85.311827,"(35.028397, -85.311827)",12:42:00 AM,01/24/2017,yes,no,no
17-010634,6500 E. BRAINERD RD.,Chattanooga,TN,37421,C4,000,Misc Report,NaN,NaN,35.018729,-85.188914,"(35.018729, -85.188914)",10:00:00 PM,01/30/2017,no,no,yes


### Save Cleaned database 

In [113]:
df_c.to_csv('cpd_cleaned.csv')

<a id='analysis'></a>
### Analysis

- Do analysis on each zone, or the 3 main zones
- Analysis of 'Incident_Type' = Part 1 Crimes
- Analyse by zipcode:
    - Part 1 Crimes per month
    - part 1 crimes by time of day